In [5]:
import pdfplumber
import pandas as pd
import re

with pdfplumber.open('/Users/henriuzas/Desktop/Kursai/Python/16627747-5.pdf') as invoice:
    for page in invoice.pages:
        extraction = page.extract_text()

#FORM FIELD DATA EXTRACTION

purchase_order_number = r'PO#(\d{5})'
vendor_name = r'(.+?)\s+Telefon'
legal_entity_billed = r'Leveransadress (\s*(.+))'
invoice_number = r'Fakturanr (\d{8})'
invoice_date = r'Fakturadatum (\d{4}-\d{2}-\d{2})'
payment_terms = r'Förfallodatum (\d{4}-\d{2}-\d{2})'
currency = 'SEK' #STATIC VALUE
invoice_total = r'(\d{1,3}(?:\s\d{3})*(?:,\d{2})?)\s+SEK'
subtotal = None #TABULAR VALUE
vat_tax_amount = None #TABULAR VALUE
tax_percentage = None #TABULAR VALUE
shipping_amount = None #NON EXISTENT
rounding_amount = r'Summa exkl\. moms\n(.+)'
account_number = r'Kundnr (\d+)'
item_description = None #UNNECESSARY
quantity = None #UNNECESSARY
item_total = r'Summa exkl\. moms\n(.+)' #MOMS.BELOPP
item_tax =  r'Summa exkl\. moms\n(.+)' #PERCENTAGE
item_tax_amount = r'Summa exkl\. moms\n(.+)' #MOMSBELOPP

purchase_order_number_match = re.search(purchase_order_number, extraction)
if purchase_order_number_match:
    purchase_order_number_value = purchase_order_number_match.group(1)
else:
    purchase_order_number_value = None

vendor_name_match = re.search(vendor_name, extraction)
if vendor_name_match:
    vendor_name_value = vendor_name_match.group(1)
else:
    vendor_name_value = None

legal_entity_billed_match = re.search(legal_entity_billed, extraction)
if legal_entity_billed_match:
    legal_entity_billed_value = legal_entity_billed_match.group(1)
else:
    legal_entity_billed_value = None

invoice_number_match = re.search(invoice_number, extraction)
if invoice_number_match:
    invoice_number_value = invoice_number_match.group(1)
else:
    invoice_number_value = None

invoice_date_match = re.search(invoice_date, extraction)
if invoice_date_match:
    invoice_date_value = invoice_date_match.group(1)
else:
    invoice_date_value = None

payment_terms_match = re.search(payment_terms, extraction)
if payment_terms_match:
    payment_terms_value = payment_terms_match.group(1)
else:
    payment_terms_value = None

invoice_total_match = re.search(invoice_total, extraction)
if invoice_total_match:
    invoice_total_value = invoice_total_match.group(1)
else:
    invoice_total_value = None

rounding_amount_match = re.findall(rounding_amount, extraction)
float_values = []
for item in rounding_amount_match:
    numeric_values = item.split()
float_values.extend(numeric_values)
float_values = [float(x.replace(',', '.')) for x in float_values]
for match in float_values:
    if (match) < 1:
        pass

account_number_match = re.search(account_number, extraction)
if account_number_match:
    account_number_value = account_number_match.group(1)
else:
    account_number_value = None

#FORM FIELD DATA STORAGE

purchase_order_number_list = []
vendor_name_list = []
legal_entity_billed_list = []
invoice_number_list = []
invoice_date_list = []
payment_terms_list = []
currency_list = []
invoice_total_list = []
rounding_amount_list = []
account_number_list = []

purchase_order_number_match = re.search(purchase_order_number, extraction)
if purchase_order_number_match:
    purchase_order_number_list.append(purchase_order_number_match.group(1))

vendor_name_match = re.search(vendor_name, extraction)
if vendor_name_match:
    vendor_name_list.append(vendor_name_match.group(1))

legal_entity_billed_match = re.search(legal_entity_billed, extraction)
if legal_entity_billed_match:
    legal_entity_billed_list.append(legal_entity_billed_match.group(1))

invoice_number_match = re.search(invoice_number, extraction)
if invoice_number_match:
    invoice_number_list.append(invoice_number_match.group(1))

invoice_date_match = re.search(invoice_date, extraction)
if invoice_date_match:
    invoice_date_list.append(invoice_date_match.group(1))

payment_terms_match = re.search(payment_terms, extraction)
if payment_terms_match:
    payment_terms_list.append(payment_terms_match.group(1))

currency_list.append(currency)

invoice_total_match = re.search(invoice_total, extraction)
if invoice_total_match:
    invoice_total_list.append(invoice_total_match.group(1))

rounding_amount_matches = re.finditer(rounding_amount, extraction)
for match in rounding_amount_matches:
     correction = match.group(1)
     n_values = correction.split()
     n_values = [float(value.replace(',', '.')) for value in n_values]
     rounding_amount_list.extend([value for value in n_values if value < 1])

account_number_match = re.search(account_number, extraction)
if account_number_match:
    account_number_list.append(account_number_match.group(1))

non_tabular_data = {
    'Purchase Order Number': purchase_order_number_list,
    'Vendor Name': vendor_name_list,
    'Legal Entity Billed': legal_entity_billed_list,
    'Invoice Number': invoice_number_list,
    'Invoice Date': invoice_date_list,
    'Payment Terms': payment_terms_list,
    'Currency': currency_list,
    'Invoice Total': invoice_total_list,
    'Rounding Amount': rounding_amount_list,
    'Account Number': account_number_list
}

form_field_df = pd.DataFrame(non_tabular_data)
form_field_df = form_field_df.T

print(form_field_df)

#TABLE FIELD DATA EXTRACTION

table_row_start_pattern = r'Moms %'

item_total_list = []
item_tax_list = []
item_tax_amount_list = []

table_area = page.crop((198, 411, 566, 435))
table_text = table_area.extract_text()
table_rows = re.split(table_row_start_pattern, table_text)[1:]

swedish_vat_rates = [6, 12, 25]

for row in table_rows:
    item_tax_match = re.findall(item_tax, row)
float_values = []
for item in item_tax_match:
    numeric_values = item.split()
float_values.extend(numeric_values)
float_values = [float(i.replace(',', '.')) for i in float_values]
for match in float_values:
    if isinstance(match, float) and match.is_integer() and match in swedish_vat_rates:
        print(int(match))

# tabular_data = {
#     'item_total': item_total_list,
#     'item_tax': item_tax_list,
#     'item_tax_amount': item_tax_amount_list
# }

# tabular_df = pd.DataFrame(tabular_data)
# print(tabular_df)

                                              0
Purchase Order Number                     77248
Vendor Name                   Avarn Security AB
Legal Entity Billed    Programavimo mokykla UAB
Invoice Number                         16627747
Invoice Date                         2023-10-12
Payment Terms                        2023-11-11
Currency                                    SEK
Invoice Total                         24 950,00
Rounding Amount                            0.31
Account Number                          2179969
25
